# Regions and Resuspension Signal

In [ ]:
import cmocean.cm as cm
import copy
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import matplotlib.gridspec as gridspec
import pandas as pd
from matplotlib.colors import LogNorm
from matplotlib.colors import ListedColormap
#

In [ ]:
filename = '/home/vvalenzuela/MOAD/Ocean_Parcels/results/Simulations_runs/PBDE_particles_for_0112022_run_365_days_full_kernels_tau_0_01_ratio_0_1_sv_5_AD_0_052.zarr'

In [ ]:
import Regions_functions_V2
from Regions_functions_V2 import plot_vertical_concentration_state_profiles, plot_vertical_total_state_profiles, plot_vertical_state_status, interpolate_volume_profile
polygon_dict = Regions_functions_V2.polygon_definition(filename)

In [ ]:
regions_water, regions_sedimented = Regions_functions_V2.status_states_regions_map(polygon_dict)
####### regions ########
# written like N1_sedimented_lon, N1_sedimented_lat 
#
region_names = ['N1', 'N2', 'N3', 'C1', 'S1', 'SP', 'HW1', 'F1', 'S2', 'H1', 'J1']

water_regions_lon = [regions_water[region]['lon'] for region in region_names]
water_regions_lat = [regions_water[region]['lat'] for region in region_names]
water_regions_depth = [regions_water[region]['depth'] for region in region_names]

sedimented_regions_lon = [regions_sedimented[region]['lon'] for region in region_names]
sedimented_regions_lat = [regions_sedimented[region]['lat'] for region in region_names]
sedimented_regions_depth = [regions_sedimented[region]['depth'] for region in region_names]

In [ ]:
data = xr.open_dataset(filename, engine='zarr')

Lets see for 1 day cycle    :o

In [ ]:
times_day = data.isel(obs = np.arange(1216, 1460)).time[0,:].values
data_day =  data.isel(obs = np.arange(1216, 1460))

In [ ]:
data_day_water = data_day.where((data_day.status < 10) & (data_day.status > 0))
data_day_sediment = data_day.where(data_day.status > 10)

In [ ]:
begin = pd.to_datetime(times_day[0]).to_pydatetime()
end = pd.to_datetime(times_day[-1]).to_pydatetime()
t_len = (end - begin).days

In [ ]:
import os; import glob; from datetime import timedelta
def date_files(date, path, prefix_type= 'grid_T.nc'):
    folder = date.strftime("%d%b%y").lower()
    prefix = os.path.join(path, folder + f'/SalishSea_1h_{date.strftime("%Y%m%d").lower()}_{date.strftime("%Y%m%d").lower()}_{prefix_type}*')
    file_name = glob.glob(prefix) 
    return file_name
#
def get_timestamps(start,length):
    timestamps=[]
    duration = timedelta(days=length)
    for day in range(duration.days):
        timestamps.append([start + timedelta(days=day)])
    return timestamps
#
dates = get_timestamps(begin,t_len)
hourly_dates = pd.date_range(start=begin, end=end, freq='H')[:-1]
#
path_days_h = '/results2/SalishSea/nowcast-green.202111/'
#
files_U = []
files_V = []
files_e3t = []
for i in range(t_len):
    files_U.append(date_files(dates[i][0],path_days_h, prefix_type = 'grid_U.nc'))
    files_V.append(date_files(dates[i][0],path_days_h, prefix_type = 'grid_V.nc'))
    files_e3t.append(date_files(dates[i][0],path_days_h, prefix_type = 'grid_T.nc'))
#
path_bat = '/ocean/vvalenzuela/MOAD/grid2/mesh_mask202108_TDV.nc'
bat_file = xr.open_dataset(path_bat)
mbathy = bat_file['mbathy'][0]
#     

In [ ]:
jjii = xr.open_dataset('/ocean/vvalenzuela/MOAD/grid/grid_from_lat_lon_mask999.nc')
def finder(lati,loni):
    j = [jjii.jj.sel(lats=lati, lons=loni, method='nearest').item()][0]
    i = [jjii.ii.sel(lats=lati, lons=loni, method='nearest').item()][0]
    return j,i
#################################################################################################    
a, b = finder(49.2,-123.5)

In [ ]:
filepaths_U = [f[0] for f in files_U]
filepaths_V = [f[0] for f in files_V]
filepaths_e3t = [f[0] for f in files_e3t]

In [ ]:
depth_idx = mbathy[a,b].values

In [ ]:
U_vels = []
for file in filepaths_U:
    datito_u = xr.open_dataset(file)['vozocrtx'].isel(depthu=depth_idx - 1, y=a, x=b)
    U_vels.append(datito_u)
#
V_vels = []
for file in filepaths_V:
    datito_v = xr.open_dataset(file)['vomecrty'].isel(depthv=depth_idx - 1, y=a, x=b)
    V_vels.append(datito_v)
#
e3ts = []
for file in filepaths_e3t:
    datito_e3t = xr.open_dataset(file)['e3t'].isel(deptht=depth_idx - 1, y=a, x=b)
    e3ts.append(datito_e3t)  
#
sshs = []
for file in filepaths_e3t:
    datito_ssh = xr.open_dataset(file)['sossheig'].isel(y=a, x=b)
    sshs.append(datito_ssh)           

In [ ]:
from Calculations_Functions import proportions_from_filename

In [ ]:
proportions = proportions_from_filename(filename = filename)
#
water, sediment =  proportions['Sewage Water'] + proportions['Colloidal Water'] + proportions['Marine Water'] , proportions['Sewage Sediment'] + proportions['Colloidal Sediment'] + proportions['Marine Sediment']

In [ ]:
U_velocity = xr.concat(U_vels, dim='time_counter')
V_velocity = xr.concat(V_vels, dim='time_counter')


In [ ]:
e3t_values = xr.concat(e3ts, dim='time_counter')
ssh_values = xr.concat(sshs, dim='time_counter')

$$u^{*} = \frac{U \cdot k}{ln(\frac{z}{z_{*}})}$$


In [ ]:
z = e3t_values / 2
z_star = 0.07
k = 0.42
#
vel_horizontal = np.sqrt(U_velocity**2 + V_velocity**2)
# Final u* calculation
with np.errstate(divide='ignore', invalid='ignore'):
    u_star = (vel_horizontal * k) / np.log(z / z_star)

In [ ]:
from scipy.interpolate import UnivariateSpline
#
spline_ustar = UnivariateSpline(range(len(u_star)), u_star.values, s=.006)
smoothed_values_ustar = spline_ustar(range(len(u_star)))
#
smoothed_ustar = pd.Series(smoothed_values_ustar, index=U_velocity.time_counter)
#
spline_ssh = UnivariateSpline(range(len(ssh_values)), ssh_values.values, s=700)
smoothed_values_ssh = spline_ssh(range(len(ssh_values)))
#
smoothed_ssh = pd.Series(smoothed_values_ssh, index=ssh_values.time_counter)
#
plt.rcParams.update({'font.size': 18})
#
fig, ax = plt.subplots(2,1,figsize = (18,8), height_ratios=[1, .5], sharex=True)
#
ax[1].plot(smoothed_ssh.index, ssh_values, 'g', alpha = .2)
ax[1].plot(smoothed_ssh, 'g')
#
ax[1].grid(alpha = .7)
ax[1].set_ylabel('SSH [m]')
#
ax[0].plot(smoothed_ustar.index, u_star, 'k', alpha = .2, label = r'Raw $\tau$')
ax[0].plot(smoothed_ustar, 'k', alpha = 1, label = r'Smoothed $\tau$')
#
#ax.vlines(x = times_day[t], ymin = 0, ymax = .015, color = 'g')
#ax.hlines(y=0.01, xmin = U_velocity['time_counter'][0], xmax = U_velocity['time_counter'][-1], color = 'g', linestyle = '--')
ax[0].set_ylabel(r'Bottom Stress due to Currents ($\tau$)')
ax[0].legend(loc = 'upper left')
ax[0].grid(alpha = .7)
#
axis = ax[0].twinx()
axis.plot(water, 'b', label = 'Water Column')
axis.plot(sediment, 'r', label = 'Sediment')
axis.set_xlim([times_day[0], times_day[-1]])
axis.legend(loc = 'upper right')
axis.set_ylabel('Proportion (%)')
#
ax[0].set_ylim([-0.001, 0.016])
axis.set_ylim([-5, 80])
#
plt.subplots_adjust(hspace=0, top=0.97, bottom=0.07, left=0.07, right=0.93)
#
#
plt.tight_layout()

Can we know when resuspension happens in the data? 

In [ ]:
status = data.status.values
#
status_now = status[:, :-1]     # current timestep
status_next = status[:, 1:]     # next timestep
#
all_ss_to_c = (status_now == 11) & (status_next == 2)
all_cs_to_c = (status_now == 12) & (status_next == 2)
all_ms_to_c = (status_now == 13) & (status_next == 2)

all_ss_to_m = (status_now == 11) & (status_next == 3)
all_cs_to_m = (status_now == 12) & (status_next == 3)
all_ms_to_m = (status_now == 13) & (status_next == 3)

total_sediment_transitions = (
    np.count_nonzero(all_ss_to_c) +
    np.count_nonzero(all_cs_to_c) +
    np.count_nonzero(all_ms_to_c) +
    np.count_nonzero(all_ss_to_m) +
    np.count_nonzero(all_cs_to_m) +
    np.count_nonzero(all_ms_to_m)
)

# Percentage of sediment transitions
print(np.count_nonzero(all_ss_to_c) / total_sediment_transitions * 100, '% ss_to_c')
print(np.count_nonzero(all_cs_to_c) / total_sediment_transitions * 100, '% cs_to_c')
print(np.count_nonzero(all_ms_to_c) / total_sediment_transitions * 100, '% ms_to_c')
print(np.count_nonzero(all_ss_to_m) / total_sediment_transitions * 100, '% ss_to_m')
print(np.count_nonzero(all_cs_to_m) / total_sediment_transitions * 100, '% cs_to_m')
print(np.count_nonzero(all_ms_to_m) / total_sediment_transitions * 100, '% ms_to_m')


In [ ]:
labels_res = ['S. S. to C.', 'C. S. to C.', 'M. S. to C.', 'S. S. to M.', 'S. C. to M.', 'S. M to M.']
per_res = [np.count_nonzero(all_ss_to_c) / total_sediment_transitions * 100, 
           np.count_nonzero(all_cs_to_c) / total_sediment_transitions * 100,
           np.count_nonzero(all_ms_to_c) / total_sediment_transitions * 100,
           np.count_nonzero(all_ss_to_m) / total_sediment_transitions * 100,
           np.count_nonzero(all_cs_to_m) / total_sediment_transitions * 100,
           np.count_nonzero(all_ms_to_m) / total_sediment_transitions * 100]
colors_res = ['k', 'r', 'g', 'k', 'r', 'g']

In [ ]:
z = data.z.values  

z_now = z[:, :-1] 


depths_ss_to_c = z_now[all_ss_to_c]
depths_cs_to_c = z_now[all_cs_to_c]
depths_ms_to_c = z_now[all_ms_to_c]

depths_ss_to_m = z_now[all_ss_to_m]
depths_cs_to_m = z_now[all_cs_to_m]
depths_ms_to_m = z_now[all_ms_to_m]

#
depth_data = [
    depths_ss_to_c, depths_cs_to_c, depths_ms_to_c,
    depths_ss_to_m, depths_cs_to_m, depths_ms_to_m
]
means = []
medians = []
mins = []
maxs = []
counts = []

for label, arr in zip(labels_res, depth_data):
    if arr.size > 0:
        means.append(arr.mean())
        medians.append(np.median(arr))
        mins.append(arr.min())
        maxs.append(arr.max())
        counts.append(arr.size)
        print(f'{label}: mean={arr.mean():.2f}, median={np.median(arr):.2f}, min={arr.min():.2f}, max={arr.max():.2f}, n={arr.size}')
    else:
        means.append(np.nan)
        medians.append(np.nan)
        mins.append(np.nan)
        maxs.append(np.nan)
        counts.append(0)
        print(f'{label}: No transitions')
#


In [ ]:
plt.rcParams.update({'font.size': 20})
#
fig, ax = plt.subplots(2,1,figsize=(20, 12), height_ratios=[1, .8], sharex=True)
bars1 = ax[0].bar(labels_res[:3], per_res[:3], color = colors_res[:3], hatch = '//', alpha = .7, label = 'To Colloidal from Sediment')
bars2 = ax[0].bar(labels_res[3:], per_res[3:], color = colors_res[3:], alpha = .7, label = 'To M. Particle from Sediment')


for bar, value in zip(bars1, per_res[:3]):
    ax[0].text(
        bar.get_x() + bar.get_width() / 2,
        bar.get_height() + 0.5, 
        f'{value:.2f}%',
        ha='center', va='bottom', fontsize=16, fontweight='bold'
    )
    #
for bar, value in zip(bars2, per_res[3:]):
    ax[0].text(
        bar.get_x() + bar.get_width() / 2,
        bar.get_height() + 0.5, 
        f'{value:.2f}%',
        ha='center', va='bottom', fontsize=16, fontweight='bold'
    )

ax[0].set_title('Resuspension Transitions vs Total Number of Sediment Resuspension Transitions', fontsize=22)
ax[0].set_ylabel('Percentage from \n Total Transtions (%)', fontsize=14)
ax[0].set_ylim(0, max(per_res) * 1.2) 
ax[0].grid(axis='y', linestyle='--', alpha=0.5)
ax[0].legend()
######
ax[1].plot(labels_res, means, '-ok', label = 'Mean')
ax[1].plot(labels_res, medians, '-db', label = 'Median')
ax[1].plot(labels_res, mins, '--sm', label = 'Min')
ax[1].plot(labels_res, maxs, '--sc', label = 'Max')
ax[1].invert_yaxis()
ax[1].grid()
ax[1].legend()
ax[1].set_ylabel('Resuspension transition Depth [m]')

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()